# Import Libraries

In [27]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Initialization

In [28]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("scs")
namespaces = ["Kickoff Meeting", "Project Meeting"]

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(temperature=0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

# Query

In [29]:
query = "Who are present in the meeting?"

# Generate Embeddings

In [30]:
def get_embeddings(text):
    """
    This function returns a list of the embeddings for a given query
    """
    text_embeddings = EMBEDDINGS.embed_query(text)
    print("Generating Embeddings: Done!")
    return text_embeddings

query_embeddings = get_embeddings(text=query)
print(query_embeddings)

Generating Embeddings: Done!
[-0.025854891166090965, 0.04573911055922508, 0.06067929416894913, -0.0012216571485623717, 0.002902596490457654, 0.023099197074770927, 0.0033348621800541878, 0.02627364732325077, -0.03876882791519165, -0.03574296832084656, 0.05635663866996765, -0.03344655781984329, -0.020802786573767662, -0.004869743250310421, -0.024409502744674683, 0.00282154674641788, -0.02850251831114292, -0.00888846255838871, 0.01683134399354458, 0.04522579535841942, 0.004413838032633066, 0.063218854367733, -0.020113863050937653, 0.033554624766111374, -0.04514474794268608, 0.023626020178198814, -0.055438071489334106, 0.01098900381475687, 0.0477113239467144, -0.009536861442029476, 0.006308377254754305, -0.02612505666911602, 0.01856040768325329, -0.004697512369602919, -0.055032823234796524, 0.008591280318796635, -0.022802013903856277, -0.005254729650914669, -0.03574296832084656, 0.0006733044283464551, 0.023099197074770927, -0.04660364240407944, -0.04790044203400612, -0.006956775672733784, 

# Get Namespace

### Semantic Similarity

In [31]:
def get_most_similar_namespace(query, namespaces, threshold=0.05):
    """
    Rank namespaces by semantic similarity to the query.
    """
    # Get query embeddings
    query_embeddings = get_embeddings(query)
    print(query_embeddings)

    # Get embeddings for each namespace in list
    namespace_embeddings = {ns: get_embeddings(ns) for ns in namespaces}
    print(namespace_embeddings)

    # Compute similarities
    similarities = {
        ns: cosine_similarity([query_embeddings], [embedding])[0][0] for ns, embedding in namespace_embeddings.items()
    }
    print(similarities.items)

    # Rank namespaes by similarity score
    ranked_namespaces = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    print(ranked_namespaces)

    # Check if the top two are close in similarity
    top_two = ranked_namespaces[:2]
    print(top_two)
    if len(top_two) > 1 and abs(top_two[0][1] - top_two[1][1]) < threshold:
        return None, top_two # Ambiguous case, return for user clarification
    
    return ranked_namespaces[0][0], ranked_namespaces # Returns most similar namespace and the ranked_namespaces with its embeddings

namespace, ranked = get_most_similar_namespace(query, namespaces)

Generating Embeddings: Done!
[-0.025877047330141068, 0.045700352638959885, 0.06064553186297417, -0.0012212209403514862, 0.002945794491097331, 0.023066380992531776, 0.0033325988333672285, 0.026268919929862022, -0.03880881518125534, -0.03580896928906441, 0.056267376989126205, -0.03340368717908859, -0.020836766809225082, -0.0048882621340453625, -0.024417662993073463, 0.0028748521581292152, -0.02853907272219658, -0.00892521720379591, 0.016823459416627884, 0.045240916311740875, 0.004408556967973709, 0.06318593770265579, -0.020093562081456184, 0.0335388146340847, -0.04515983909368515, 0.02360689453780651, -0.05551065877079964, 0.010992677882313728, 0.04764619842171669, -0.009553562849760056, 0.00636115949600935, -0.026133790612220764, 0.018512561917304993, -0.00465178769081831, -0.05505122244358063, 0.008567127399146557, -0.02280963771045208, -0.005232838913798332, -0.03575491905212402, 0.000726314028725028, 0.02313394658267498, -0.04667327553033829, -0.04783537983894348, -0.0069388323463499

### User Clarification Loop

In [32]:
def clarify_with_user(ambiguous_namespaces):
    """
    Ask the user to clarify when multiple namespaces are similar.
    """
    options = [ns[0] for ns in ambiguous_namespaces]
    print(options)
    print(f"Did you mean:\n1. {options[0]}\n2. {options[1]}")

    # Simulate user input for demonstration
    user_choice = int(input("Please choose 1 or 2: "))-1
    return options[user_choice]

### Integration

In [33]:
def resolve_namespace(query, namespaces):
    """
    Resolves the namespace by either selecting the most similar one or prompting the user for clarification.
    """
    namespace, ranked = get_most_similar_namespace(query, namespaces)
    print(namespaces, ranked)

    if namespace:
        print(f"Selected namespace: {namespace}")
        return namespace
    else:
        print("Ambiguity detected!")
        return clarify_with_user(ranked)
    
resolved_namespace = resolve_namespace(query, namespaces)
print(f"Namespace: {resolved_namespace}")

Generating Embeddings: Done!
[-0.025877047330141068, 0.045700352638959885, 0.06064553186297417, -0.0012212209403514862, 0.002945794491097331, 0.023066380992531776, 0.0033325988333672285, 0.026268919929862022, -0.03880881518125534, -0.03580896928906441, 0.056267376989126205, -0.03340368717908859, -0.020836766809225082, -0.0048882621340453625, -0.024417662993073463, 0.0028748521581292152, -0.02853907272219658, -0.00892521720379591, 0.016823459416627884, 0.045240916311740875, 0.004408556967973709, 0.06318593770265579, -0.020093562081456184, 0.0335388146340847, -0.04515983909368515, 0.02360689453780651, -0.05551065877079964, 0.010992677882313728, 0.04764619842171669, -0.009553562849760056, 0.00636115949600935, -0.026133790612220764, 0.018512561917304993, -0.00465178769081831, -0.05505122244358063, 0.008567127399146557, -0.02280963771045208, -0.005232838913798332, -0.03575491905212402, 0.000726314028725028, 0.02313394658267498, -0.04667327553033829, -0.04783537983894348, -0.0069388323463499

# Metadata Filtering

## Fuzzy Match

In [34]:
def fuzzy_match(title1, title2, threshold=80):
    """
    Perform a fuzzy match between two titles using RapidFuzz.
    Returns True if the similarity score is above the threshold.
    """
    similarity_score = fuzz.partial_ratio(title1.lower(), title2.lower())
    return similarity_score >= threshold

## Extract Metadata

In [35]:
def extract_metadata_from_query(query):
  prompt = f"""
  You are a helpful assistant. Extract the meeting title and the meeting date from the following query.
  If the meeting title or date is not explicitly mentioned, return 'unknown'.
  If the date is mentioned as word, it should be formatted as 'YYYY-MM-DD'

  Query: {query}

  Provide the meeting title and date as a Python dictionary in this format:
  {{"meeting_title": "title_here", "date": "date_here"}}
  """

  response = LLM.invoke(prompt)
  metadata_str = response.content.strip()
  metadata_dict = ast.literal_eval(metadata_str)
  return metadata_dict

metadata = extract_metadata_from_query(query)
print(metadata)

SyntaxError: invalid syntax (<unknown>, line 1)

# Query Pinecone Index

In [47]:
def query_pinecone_index(query_embeddings, meeting_title, date, top_k=3, include_metadata=True):
    """
    Query a Pinecone index.
    """
    # Build filter conditions directly for Pinecone
    filter_conditions = {}

    # Query Pinecone using the build filter conditions
    query_response = index.query(
        vector=query_embeddings,
        filter=filter_conditions,
        top_k=top_k,
        include_metadata=include_metadata,
        namespace=meeting_title) # Filter based on metadata

    print("Querying Pinecone Index: Done!")
    return query_response

answers = query_pinecone_index(query_embeddings=query_embeddings, meeting_title=resolved_namespace, date="unknown")
print(answers)

Querying Pinecone Index: Done!
{'matches': [{'id': 'e773312289360121c6069407784bbc732f6f8d4505330eb28deaaa0add6a0cd7',
              'metadata': {'date': '2024-12-01',
                           'text': 'Czech: Hello my name is Czech.\n'
                                   'Gian: Hello my name is Gian.\n'
                                   'Shaundyl: Hello my name is Shaundyl.\n'
                                   'Czech (Team Lead): Alright, everyone, '
                                   'thanks for joining today’s meeting. We '
                                   'have about 10 minutes to go over the final '
                                   "details before the product launch. Let's "
                                   'start with the progress update. Bob, how '
                                   'are we doing on the development front?',
                           'title': 'Project Meeting'},
              'score': 0.434614152,
              'values': []},
             {'id': 'ae52e04

# Combining Text from Multiple Document Matches

In [48]:
text_answer = " ".join([doc['metadata']['text'] for doc in answers['matches']])
print(text_answer)

Czech: Hello my name is Czech.
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front? Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.
Czech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?
Shaundyl: Nothing from my side. I’ll update you if any blockers come up.
Gian: I’m all set. Let’s get this done! Czech: Great to hear that. Gian, how are we looking on the project timeline? Any changes or concerns?
Gian (Project Manager): We’re on track, but barely. The marketing materials were delayed by two d

## Prompt

In [49]:
prompt = f"""You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: {text_answer}
        Question: {query}"""

print(prompt)

You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: Czech: Hello my name is Czech.
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front? Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.
Czech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?


# LLM

In [50]:
def better_query_response(prompt):
    """
    This function returns a better response using LLM
    """
    better_answer = LLM.invoke(prompt)
    print("Generating Better Response: Done!")
    return better_answer

final_answer = better_query_response(prompt=prompt)
print(final_answer.content)

Generating Better Response: Done!
Czech, Gian, and Shaundyl are present in the meeting.


# Chatbot Response

In [51]:
def Chatbot(query, meeting_title=None, date=None):
    print(query)
    meeting_title = resolve_namespace(query, namespaces)
    query_embeddings = get_embeddings(text=query)

    answers = query_pinecone_index(
        query_embeddings=query_embeddings,
        meeting_title=meeting_title,
        date=date
        )
    print(answers)

    text_answers = " ".join([doc['metadata']['text'] for doc in answers['matches']])
    print(text_answers)
    final_answer = better_query_response(prompt=prompt)
    return final_answer.content

response = Chatbot(query)
print(response)

Who are present in the meeting?
Generating Embeddings: Done!
[-0.025877047330141068, 0.045700352638959885, 0.06064553186297417, -0.0012212209403514862, 0.002945794491097331, 0.023066380992531776, 0.0033325988333672285, 0.026268919929862022, -0.03880881518125534, -0.03580896928906441, 0.056267376989126205, -0.03340368717908859, -0.020836766809225082, -0.0048882621340453625, -0.024417662993073463, 0.0028748521581292152, -0.02853907272219658, -0.00892521720379591, 0.016823459416627884, 0.045240916311740875, 0.004408556967973709, 0.06318593770265579, -0.020093562081456184, 0.0335388146340847, -0.04515983909368515, 0.02360689453780651, -0.05551065877079964, 0.010992677882313728, 0.04764619842171669, -0.009553562849760056, 0.00636115949600935, -0.026133790612220764, 0.018512561917304993, -0.00465178769081831, -0.05505122244358063, 0.008567127399146557, -0.02280963771045208, -0.005232838913798332, -0.03575491905212402, 0.000726314028725028, 0.02313394658267498, -0.04667327553033829, -0.047835